# <b> Calculation </b>
___

<b> import modules </b>

In [14]:
import pandas as pd
import numpy as np

<b> read recoded csv file to a pandas dataframe </b>

In [15]:
df = pd.read_csv("df_recoded.csv")

## Feature Engineering
___

### Deal with Highly Correlated Features
_Note_: the <b> _first option_ </b> is to only select one of the highly pairwise-correlated features.
<br> the <b> _second option_ </b> is to combine them into a single feature and see in feature selection which one is the best predictor -> either one of the highly correlated features or the combined feature.

### 1. Dealing with Continuous Features

_left align all markdown tables_

In [16]:
%%html
<style>
    table {
        display: inline-block
    }
</style>


#### <b> Table 1: </b> Absolute Pairwise correlations (>|0.5|)
##### (Of Original Dataset)

| Feature 1      | Feature 2                | Correlation |
|----------------|--------------------------|-------------|
| BounceRates    | ExitRates                | 0\.913004   |
| ProductRelated | ProductRelated\_Duration | 0\.860927   |
| Informational  | Informational\_Duration  | 0\.618955   |
| Administrative | Administrative\_Duration | 0\.601583   |

<b> 1.1 Combine BounceRates & ExitRates 
> We combine them in the following ways:
    - Taking the average (x1+x2)/2
    - Adding both values (x1+x2)
    - Dividing both values (x1/x2)
    - Percentage Change ((x1-x2)/x2)

Average

In [17]:
# combine BounceRates and ExitRates by taking the average row-wise
df['avg_exit_bounce_rates'] = df[['BounceRates', 'ExitRates']].mean(axis=1)

# test if it worked
# df[['ExitRates','BounceRates', 'avg_exit_bounce_rates']].head()

Addition

In [18]:
# combine BounceRates and ExitRates by addition row-wise
df['add_exit_bounce_rates'] = df['BounceRates']+df['ExitRates']

# test if it worked
#df[['ExitRates','BounceRates', 'add_exit_bounce_rates']].head()

Division

In [19]:
# combine BounceRates and ExitRates by division row-wise
df['div_exit_bounce_rates'] = df['BounceRates']/df['ExitRates']

# test if it worked
#df[['ExitRates','BounceRates', 'div_exit_bounce_rates']].head()
#df[['ExitRates','BounceRates', 'div_exit_bounce_rates']].describe()
#df[['ExitRates','BounceRates', 'div_exit_bounce_rates']].isna().sum()

Percentage Change

In [20]:
# combine BounceRates and ExitRates by Percentage Change row-wise
df['perc_exit_bounce_rates'] = ((df['BounceRates']-df['ExitRates'])/df['ExitRates'])

# test if it worked
#df[['ExitRates','BounceRates', 'perc_exit_bounce_rates']].head()
#df[['ExitRates','BounceRates', 'perc_exit_bounce_rates']].describe()
#df[['ExitRates','BounceRates', 'perc_exit_bounce_rates']].isna().sum()

<b> 1.2 Combine Administrative & Administrative_Duration </b>
<br> We combine them by taking the Administrative Duration per Page

In [21]:
df['Administrative_Duration_pp'] = df['Administrative_Duration']/df['Administrative']

# test if it worked
#df[['Administrative_Duration', 'Administrative', 'Administrative_Duration_pp']]
# Note: results in NaN, should be replaced by zero -> will be imputed

# only show df where there is a Administrative Duration larger than zero
#df[df['Administrative_Duration'] > 0][['Administrative_Duration', 'Administrative', 'Administrative_Duration_pp']].head(5)
# conclusion: it now correctly has a column with the Administrative Duration per page

<b> 1.3 Combine Informational & Informational_Duration </b>
<br> We combine them by taking the Information Duration per Page

In [22]:
df['Informational_Duration_pp'] = df['Informational_Duration']/df['Informational']

# test if it worked
#df[['Informational_Duration', 'Informational', 'Informational_Duration_pp']].head()
# Note: results in NaN, should be replaced by zero -> will be imputed

# only show df where there is a informational duration larger than zero
#df[df['Informational_Duration'] > 0][['Informational_Duration', 'Informational', 'Informational_Duration_pp']].head(5)
# conclusion: it now correctly has a column with the Informational Duration per page

 <b> 1.4 Combine ProductRelated & ProductRelated_Duration </b>
<br> We combine them by taking the Product-Related Duration per Page

In [23]:
df['ProductRelated_Duration_pp'] = df['ProductRelated_Duration']/df['ProductRelated']

# test if it worked
#df[['ProductRelated_Duration', 'ProductRelated', 'ProductRelated_Duration_pp']].head()
# show number of NaN's -> 38
# df[['ProductRelated_Duration', 'ProductRelated', 'ProductRelated_Duration_pp']].isna().sum()

# Note: results in NaN, should be replaced by zero -> will be imputed
# conclusion: it now correctly has a column with the ProductRelated Duration per page

# Imputation
___
<b> Check for NaN's in calculated features </b>

In [24]:
# create a function to create a dataframe with # NaN's and % NaN's
def function_nans(df):
    # Count NaN's
    list_nan = df.isna().sum()

    # Percentage of missing values for each column (To see the amount of usable data for each column and infer the effect replacing NaN's will have by some metric in part 2)
    list_nan_pct = df.isna().mean().round(4) * 100

    # create a dataframe showing the missing and percentage that is missing
    df_combined = pd.concat([list_nan.rename('# NaNs'), list_nan_pct.rename('% NaNs')], axis=1)
    
    return(df_combined)

# list of added features
features_added = ['avg_exit_bounce_rates', 'add_exit_bounce_rates', 'div_exit_bounce_rates', 'perc_exit_bounce_rates','Administrative_Duration_pp', 'Informational_Duration_pp', 'ProductRelated_Duration_pp']

# show a dataframe for number of NaN's and the % of NaN's for each added feature
function_nans(df=df[features_added])

,# NaNs,% NaNs
avg_exit_bounce_rates,0,0.00
add_exit_bounce_rates,0,0.00
div_exit_bounce_rates,76,0.62
perc_exit_bounce_rates,76,0.62
Administrative_Duration_pp,5768,46.78
Informational_Duration_pp,9699,78.66
ProductRelated_Duration_pp,38,0.31


<b> Replace NaN's by zero </b>

In [25]:
# create a function to replace NaN's by zero for a given dataframe with a given list of columnnames
def replace_nan_by_zero(df, column_list): 
    # replace nans by zeros
    df[column_list] = df[column_list].replace(np.NaN, 0)

    
# apply function to added features which have NaN's 
replace_nan_by_zero(df=df, column_list=['div_exit_bounce_rates', 'perc_exit_bounce_rates', 'Administrative_Duration_pp', 'Informational_Duration_pp', 'ProductRelated_Duration_pp'])

# test if replacement worked
function_nans(df=df[features_added])

,# NaNs,% NaNs
avg_exit_bounce_rates,0,0.0
add_exit_bounce_rates,0,0.0
div_exit_bounce_rates,0,0.0
perc_exit_bounce_rates,0,0.0
Administrative_Duration_pp,0,0.0
Informational_Duration_pp,0,0.0
ProductRelated_Duration_pp,0,0.0


### __Save Calculated Features Dataset to a CSV File__ #

In [26]:
# save a copy of the dataframe under 'df_calculation'
# define name of the csv file
df_calculation = df

file_name = 'df_calculation.csv'

# export pandas dataframe to csv
df_calculation.to_csv('../project2-mtb/' + file_name, index=False)